In [3]:
%pip install ollama

  Obtaining dependency information for ollama from https://files.pythonhosted.org/packages/31/83/c3ffac86906c10184c88c2e916460806b072a2cfe34cdcaf3a0c0e836d39/ollama-0.4.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<0.29,>=0.27 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3.0.0,>=2.9.0 from https://files.pythonhosted.org/packages/f4/3c/8cc1cc84deffa6e25d2d0c688ebb80635dfdbf1dbea3e30c541c8cf4d860/pydantic-2.10.6-py3-none-any.whl.metadata
  Obtaining dependency information for anyio from https://files.pythonhosted.org/packages/46/eb/e7f063ad1fec6b3178a3cd82d1a3c4de82cccf283fc42746168188e1cdd5/anyio-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi from https://files.pythonhosted.org/packages/a5/32/8f6669fc4798494966bf446c8c4a162e0b5d893dff088afddf76414f70e1/certifi-2024.12.14-py3-no

In [24]:
from ollama import chat
from IPython.display import Markdown, display

rule = '''
你的任务是做 javascript 代码质量检查，下面会定义一些检查规则，并列举正确案例和错误案例供你参考。
规则由规则描述、规则说明、错误案例、正确案例，四部分构成。

1. 规则描述以 Markdown H1 标题呈现。
2. 规则说明以 H2 标题呈现，可以省略。
3. 错误案例和正确案例是示例代码，以H2标题呈现，代码中的注释部分会对代码中合理和不合理的地方做出解释。

以下是定义的规则：

# 在大于 5 行的函数里，或者全局代码中，不要使用单字命名的变量。
## 错误案例
```
var l = data.length;
b = l + a; 
```

## 正确案例
```
data.map(d => d.length);
```

# 应该有且只有方法和函数应以动词开头

## 规则说明
函数的命名需要体现内部工作，应该以不包含时态的动词开头。
值变量以动词命名，则会容易让人误解为是一个匿名函数。

以下函数的命名可以不遵守此规则：
- 回调函数
- 生命周期函数
- 框架级别函数
- getter/setter 

## 错误案例
```
// 以名词开头，看不明白什么有什么功能
function option() {}

// 时态不对
function updatedTime() {}
```

## 正确案例
```
function selectOption() {}
function updateTime() {}

class App {
    // 因为是 getter 函数，不需要遵守此规则
    get name() { }
    // 因为是生命周期函数，所以不需要遵守此规则
    componentDidMount() {}
    // 因为是点击事件回调函数，不需要遵守此规则
    onModalClick() { }
}
```

# 后续
后续将会告诉你要检查的代码，你的目的是将不符合规则要求的代码输出出来，给出错误原因，并给出改进意见。

最后将对代码的检查结果以 Output 格式输出：
```
type Output = Array<{
	line: number, // 出错的代码行数
	reason: string, // 违反的规则
	fix: string, // 修改建议	
}>;
```
'''

test = '''
let a = 1;
[].map(b => a+b);
let doList = [];
class Comp{
    onMounted(){}
    makedName(b) {
        return <span>{b}</span>;
    }
    renderList() {
        var a = this.data;
        var l = a.map(b => {
            return this.makedName(b);
        });
        l = l.filter(Boolean);
        return l;
    }
    render() {
        return <div>this.renderList()</div>;
    }
}
'''
messages = [
  {
    'role': 'user',
    'content': rule,
  },
  {
    'role': 'user',
    'content': test,
  }
]

def deleteThink(original_string):
    while '<think>' in original_string:
      start_index = original_string.find('<think>')
      end_index = original_string.find('</think>') + len('</think>')
      # 切片操作，将匹配到的部分删除
      original_string = original_string[:start_index] + original_string[end_index:]
    return original_string


response = chat('deepseek-r1:14b', messages=messages, keep_alive="5m")
# display(Markdown(deleteThink(response['message']['content'])))
print(response['message']['content'])


<think>
嗯，我现在要检查一段JavaScript代码的质量，根据给定的规则。首先，我得仔细阅读用户提供的规则，然后分析每段代码是否符合这些规则。

第一个规则是在大于5行的函数里或者全局代码中，不要使用单字命名的变量。错误案例用了l和b这样的单字母变量，而正确案例则用有意义的名字或避免在长函数中使用单字母变量。

第二个规则是方法和函数必须以动词开头，除了回调、生命周期钩子、框架级别函数和getter/setter。比如选项中的function option()就不符合，应该改为selectOption。

现在看代码：

全局代码部分：
let a = 1;
[].map(b => a+b);
这里有b作为参数，是单字母变量，而且在全局代码中使用了单字命名。这违反第一个规则。所以需要将b改为有意义的名字，比如item或者value。

接下来的doList数组，名字没问题，因为是一个列表，可能存储的是任务或事项，长度合适。

然后是Comp类：
onMounted(){} — 这个方法名不符合第二个规则吗？onMounted是以动词开头的，是否属于生命周期函数？根据规则说明， 生命周期函数不需要遵守以动词开头。所以这个是对的。

makedName(b)：这里用了名词开头的方法名make，这可能有问题。因为规则要求函数必须以动词开头。比如应该改为makeName或者buildName等。

renderList()方法内部：
var a = this.data; 这里a是单字母变量，而且在5行以上的函数中使用了单字命名，违反第一个规则。需要改名为更明确的变量名，比如dataItems。

var l = a.map(...)；同样，l是单字母变量，在长函数中使用，应该改为lengths或者mappedData等。

接下来，代码中有：
l = l.filter(Boolean);
这里Boolean作为参数可能没问题，但filter后的结果可以改名为filteredList，更明确。

在render()方法中返回的内容没有问题，但内部的this.renderList()调用是正确的。

总结一下，错误点：

1. 全局map中的b => a + b：变量名太短，并且是在全局代码中使用。
2. makedName函数名以名词开头，应该改为动词开头。
3. 